In [10]:
##### Imports ######
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
import json

In [54]:
# Removes XML tags from the data
def clean_data(data):
    try:
        tags = ET.fromstring(str(data))
        return ET.tostring(tags, method='text').decode("utf-8")
    except:
        return data

# Finds all different places and removes the XML tags from them
def clean_places(places):
    placelist = []
    for i in places:
        tags = ET.fromstring(str(i))
        placelist.append(ET.tostring(tags, method='text').decode("utf-8"))
        
    return placelist

# Removes title, place and the XML tags from the text
def clean_text(text):
    for tag in text.findAll():
        tag.replaceWith('')

    return text.text.replace('\n','')

# Creates dictionary with all documents in the Reuters database, assigns doc_ID to all documents. 
def load_data():
    data_dict = {}
    data_id = 0
#     with open('reuters.json', 'w') as f:
#         json.dump('', f)
    
    for i in range(8):
        data1 = open('reuters_data/reut2-00'+ str(i) +'.sgm')
        soup = BeautifulSoup(data1,'lxml')
        items = soup.findAll('reuters')
        
        for doc in items:
            ID =     {'docID':data_id}
            data =   {'date':clean_data(doc.date),
                          'topic':clean_data(doc.topics), 
                          'place':clean_places(doc.places.findAll('d')), 
                          'people':clean_data(doc.people), 
                          'orgs':clean_data(doc.orgs), 
                          'exchanges':clean_data(doc.exchanges), 
                          'companies':clean_data(doc.companies), 
                          'title':clean_data(doc.title), 
                          'text':clean_text(doc.findAll('text')[0]) 
                      }
#             data_str = data_str + '\n' + ID + '\n' + data
            with open('reuters.json', 'r') as f:
                temp = f.read()
            with open('reuters.json', 'w') as f:
                f.write(str(temp) + '\n')
                f.write(str(ID) + '\n')
                f.write(str(data) + '\n')
                
            print('Document', data_id, 'of 8000', end='\r')
            data_id += 1
    
#     with open('reuters.json', 'w') as f:
#         json.dump(data_str, f)
    
    return 'done'

load_data()

'done'

In [ ]:
with open(DATA_FILENAME, mode='w', encoding='utf-8') as feedsjson:
    entry = {'name': args.name, 'url': args.url}
    feeds.append(entry)
    json.dump(feeds, feedsjson)